In [1]:
import os
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

from utils.paths import ROOT, DATA_DIR, RAW_DIR, PROCESSED_DIR, EQUITY_DIR

### Preprocess data

#### Read a CSV file

In [2]:
stocks_eq_pth = os.path.join(PROCESSED_DIR, "tech_stocks_us_equity_1d.csv")
stocks_eq_df = pd.read_csv(stocks_eq_pth)
# stocks_eq_df.columns = ["Date", "Close", "High", "Low", "Open", "Volume"]
stocks_eq_df.head()

/tmp/ipykernel_108526/2928653373.py:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,26

,Price,Price.1,Price.2,Close,Close.1,Close.2,Close.3,Close.4,Close.5,Close.6,...,Volume.72,Volume.73,Volume.74,Volume.75,Volume.76,Volume.77,Volume.78,Volume.79,Volume.80,Volume.81
0,Ticker,ANSS,JNPR,AAPL,ACN,ADBE,ADI,ADP,ADSK,AKAM,...,TDY,TEL,TER,TRMB,TXN,TYL,UBER,VRSN,WDC,ZBRA
1,Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-02,NaN,NaN,24.26104164123535,74.25262451171875,72.33999633789062,44.51115036010742,65.98639678955078,59.529998779296875,63.25,...,145700,948900,1030300,1106000,4020100,216200,NaN,684500,1278547,411800
3,2015-01-05,NaN,NaN,23.57756996154785,72.99890899658203,71.9800033569336,43.70171356201172,65.93885803222656,58.65999984741211,61.939998626708984,...,266200,953400,1877000,1254900,5599300,270300,NaN,874400,2199752,420300
4,2015-01-06,NaN,NaN,23.57979393005371,72.47238159179688,70.52999877929688,42.67587661743164,65.3922348022461,57.5,60.66999816894531,...,243000,1448800,2377700,2291600,5468400,239600,NaN,1289300,2665713,527500


#### Parse Columns

In [3]:
parsed_columns = []
# Loop each columns
for col_name in stocks_eq_df.columns:
    
    parts = col_name.split('.') # แยกชื่อด้วยจุด
    feature = parts[0]
    ticker = stocks_eq_df.loc[0, col_name]

    # เพิ่ม (Feature, Ticker_ID) เข้าไปใน List
    parsed_columns.append((feature, ticker))

# Applying MultiIndex
new_index = pd.MultiIndex.from_tuples(parsed_columns, names=['Feature', 'Ticker'])
stocks_eq_df.columns = new_index

stocks_eq_df.head()

Feature       Price                          Close                     \
Ticker       Ticker  ANSS  JNPR               AAPL                ACN   
0            Ticker  ANSS  JNPR               AAPL                ACN   
1              Date   NaN   NaN                NaN                NaN   
2        2015-01-02   NaN   NaN  24.26104164123535  74.25262451171875   
3        2015-01-05   NaN   NaN  23.57756996154785  72.99890899658203   
4        2015-01-06   NaN   NaN  23.57979393005371  72.47238159179688   

Feature                                                           \
Ticker                ADBE                ADI                ADP   
0                     ADBE                ADI                ADP   
1                      NaN                NaN                NaN   
2        72.33999633789062  44.51115036010742  65.98639678955078   
3         71.9800033569336  43.70171356201172  65.93885803222656   
4        70.52999877929688  42.67587661743164   65.3922348022461   

Feature                                          ...  Volume           \
Ticker                 ADSK                AKAM  ...     TDY      TEL   
0                      ADSK                AKAM  ...     TDY      TEL   
1                       NaN                 NaN  ...     NaN      NaN   
2        59.529998779296875               63.25  ...  145700   948900   
3         58.65999984741211  61.939998626708984  ...  266200   953400   
4                      57.5   60.66999816894531  ...  243000  1448800   

Feature                                                                     
Ticker       TER     TRMB      TXN     TYL  UBER     VRSN      WDC    ZBRA  
0            TER     TRMB      TXN     TYL  UBER     VRSN      WDC    ZBRA  
1            NaN      NaN      NaN     NaN   NaN      NaN      NaN     NaN  
2        1030300  1106000  4020100  216200   NaN   684500  1278547  411800  
3        1877000  1254900  5599300  270300   NaN   874400  2199752  420300  
4        2377700  2291600  5468400  239600   NaN  1289300  2665713  527500  

[5 rows x 413 columns]

#### Remove 2 rows

In [4]:
# Remove 2 rows
steq_df = stocks_eq_df.iloc[2:]
steq_df = steq_df.reset_index(drop=True)
steq_df.head()

Feature       Price                         Close                     \
Ticker       Ticker ANSS JNPR                AAPL                ACN   
0        2015-01-02  NaN  NaN   24.26104164123535  74.25262451171875   
1        2015-01-05  NaN  NaN   23.57756996154785  72.99890899658203   
2        2015-01-06  NaN  NaN   23.57979393005371  72.47238159179688   
3        2015-01-07  NaN  NaN  23.910430908203125  73.99351501464844   
4        2015-01-08  NaN  NaN  24.829124450683594   75.1218490600586   

Feature                                                           \
Ticker                ADBE                ADI                ADP   
0        72.33999633789062  44.51115036010742  65.98639678955078   
1         71.9800033569336  43.70171356201172  65.93885803222656   
2        70.52999877929688  42.67587661743164   65.3922348022461   
3        71.11000061035156  43.12467956542969    65.962646484375   
4        72.91999816894531  43.88603591918945  67.47581481933594   

Feature                                          ...  Volume           \
Ticker                 ADSK                AKAM  ...     TDY      TEL   
0        59.529998779296875               63.25  ...  145700   948900   
1         58.65999984741211  61.939998626708984  ...  266200   953400   
2                      57.5   60.66999816894531  ...  243000  1448800   
3        57.380001068115234   60.15999984741211  ...  245300  1929000   
4         58.79999923706055   61.56999969482422  ...  240300  1635300   

Feature                                                                    
Ticker       TER     TRMB      TXN     TYL UBER     VRSN      WDC    ZBRA  
0        1030300  1106000  4020100  216200  NaN   684500  1278547  411800  
1        1877000  1254900  5599300  270300  NaN   874400  2199752  420300  
2        2377700  2291600  5468400  239600  NaN  1289300  2665713  527500  
3         902400  2096400  4577100  220300  NaN   552400  2054222  467800  
4        1626800  2036000  6295200  264700  NaN  1282600  3351291  324400  

[5 rows x 413 columns]

#### Set Indexes

In [6]:
# 1. Get col name
first_col_name = steq_df.columns[0]

# 2. Covert Cols to datetime
steq_df[first_col_name] = pd.to_datetime(steq_df[first_col_name])

# 3. Set Col be Index
steq_df = steq_df.set_index(first_col_name, drop=True)

# 4. Change index name
steq_df.index.name = 'Date'

steq_df.head()

Feature    Price                    Close                     \
Ticker      ANSS JNPR                AAPL                ACN   
Date                                                           
2015-01-02   NaN  NaN   24.26104164123535  74.25262451171875   
2015-01-05   NaN  NaN   23.57756996154785  72.99890899658203   
2015-01-06   NaN  NaN   23.57979393005371  72.47238159179688   
2015-01-07   NaN  NaN  23.910430908203125  73.99351501464844   
2015-01-08   NaN  NaN  24.829124450683594   75.1218490600586   

Feature                                                              \
Ticker                   ADBE                ADI                ADP   
Date                                                                  
2015-01-02  72.33999633789062  44.51115036010742  65.98639678955078   
2015-01-05   71.9800033569336  43.70171356201172  65.93885803222656   
2015-01-06  70.52999877929688  42.67587661743164   65.3922348022461   
2015-01-07  71.11000061035156  43.12467956542969    65.962646484375   
2015-01-08  72.91999816894531  43.88603591918945  67.47581481933594   

Feature                                                                 ...  \
Ticker                    ADSK                AKAM                AMAT  ...   
Date                                                                    ...   
2015-01-02  59.529998779296875               63.25  21.807235717773438  ...   
2015-01-05   58.65999984741211  61.939998626708984  21.108293533325195  ...   
2015-01-06                57.5   60.66999816894531     20.313232421875  ...   
2015-01-07  57.380001068115234   60.15999984741211  20.505441665649414  ...   
2015-01-08   58.79999923706055   61.56999969482422   21.16071128845215  ...   

Feature     Volume                                                            \
Ticker         TDY      TEL      TER     TRMB      TXN     TYL UBER     VRSN   
Date                                                                           
2015-01-02  145700   948900  1030300  1106000  4020100  216200  NaN   684500   
2015-01-05  266200   953400  1877000  1254900  5599300  270300  NaN   874400   
2015-01-06  243000  1448800  2377700  2291600  5468400  239600  NaN  1289300   
2015-01-07  245300  1929000   902400  2096400  4577100  220300  NaN   552400   
2015-01-08  240300  1635300  1626800  2036000  6295200  264700  NaN  1282600   

Feature                      
Ticker          WDC    ZBRA  
Date                         
2015-01-02  1278547  411800  
2015-01-05  2199752  420300  
2015-01-06  2665713  527500  
2015-01-07  2054222  467800  
2015-01-08  3351291  324400  

[5 rows x 412 columns]

#### Drop Price Column

In [9]:
# filt means filterd
# steq means stock equity
steq_df = steq_df.drop(columns="Price", level="Feature")
steq_df.head()

Feature                  Close                                        \
Ticker                    AAPL                ACN               ADBE   
Date                                                                   
2015-01-02   24.26104164123535  74.25262451171875  72.33999633789062   
2015-01-05   23.57756996154785  72.99890899658203   71.9800033569336   
2015-01-06   23.57979393005371  72.47238159179688  70.52999877929688   
2015-01-07  23.910430908203125  73.99351501464844  71.11000061035156   
2015-01-08  24.829124450683594   75.1218490600586  72.91999816894531   

Feature                                                               \
Ticker                    ADI                ADP                ADSK   
Date                                                                   
2015-01-02  44.51115036010742  65.98639678955078  59.529998779296875   
2015-01-05  43.70171356201172  65.93885803222656   58.65999984741211   
2015-01-06  42.67587661743164   65.3922348022461                57.5   
2015-01-07  43.12467956542969    65.962646484375  57.380001068115234   
2015-01-08  43.88603591918945  67.47581481933594   58.79999923706055   

Feature                                                                 \
Ticker                    AKAM                AMAT                 AMD   
Date                                                                     
2015-01-02               63.25  21.807235717773438  2.6700000762939453   
2015-01-05  61.939998626708984  21.108293533325195  2.6600000858306885   
2015-01-06   60.66999816894531     20.313232421875   2.630000114440918   
2015-01-07   60.15999984741211  20.505441665649414  2.5799999237060547   
2015-01-08   61.56999969482422   21.16071128845215   2.609999895095825   

Feature                         ...  Volume                             \
Ticker                    ANET  ...     TDY      TEL      TER     TRMB   
Date                            ...                                      
2015-01-02  3.9175000190734863  ...  145700   948900  1030300  1106000   
2015-01-05   3.987499952316284  ...  266200   953400  1877000  1254900   
2015-01-06  3.9881250858306885  ...  243000  1448800  2377700  2291600   
2015-01-07   4.071249961853027  ...  245300  1929000   902400  2096400   
2015-01-08   4.132500171661377  ...  240300  1635300  1626800  2036000   

Feature                                                     
Ticker          TXN     TYL UBER     VRSN      WDC    ZBRA  
Date                                                        
2015-01-02  4020100  216200  NaN   684500  1278547  411800  
2015-01-05  5599300  270300  NaN   874400  2199752  420300  
2015-01-06  5468400  239600  NaN  1289300  2665713  527500  
2015-01-07  4577100  220300  NaN   552400  2054222  467800  
2015-01-08  6295200  264700  NaN  1282600  3351291  324400  

[5 rows x 410 columns]

#### Check feature nums

In [11]:
num_features = steq_df.columns.get_level_values("Feature").nunique()
num_tickers = steq_df.columns.get_level_values("Ticker").nunique()

print(f"Number of Features: {num_features}")
print(f"Number of Tickers: {num_tickers}")

# ---  "Name list" ---
print("\nUnique Features:")
print(steq_df.columns.get_level_values('Feature').unique())

print("\nUnique Tickers (sample):")
print(steq_df.columns.get_level_values('Ticker').unique()[:5])

Number of Features: 5
Number of Tickers: 82

Unique Features:
Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Feature')

Unique Tickers (sample):
Index(['AAPL', 'ACN', 'ADBE', 'ADI', 'ADP'], dtype='object', name='Ticker')


#### Save File for Backup

In [13]:
steq_pth = os.path.join(PROCESSED_DIR, "v1_tech_stocks_us_equity_1d.csv")
steq_df.to_csv(steq_pth)

### EDA
* steq_df = stock equity dataframe